# Lab 7 - Wyatt Madden & Dan Crowley

In [3]:
import scipy.io as scipy_io
from scipy import sparse 
import numpy as np
import quadprog
import cvxopt
from cvxopt import matrix, solvers 

In [4]:
mat = scipy_io.loadmat('cbcl1.mat')

In [9]:
#sol=solvers.lp(c,A,b)

X = mat["X"]
l = mat["L"]
    
def softsvm(X, l, gamma):
    D,N = X.shape

    x = np.repeat(1, N + D + 1) #should it be 1? i honestly dont know
    #G = np.identity(n=N+D+1) * np.concatenate((np.repeat(0.00001, N), np.repeat(1, D), np.repeat(0.00001,1)), axis = 0)
    P = np.identity(n=N+D+1) * np.concatenate((np.repeat(0, N), np.repeat(1, D), np.repeat(0,1)), axis = 0)
    q = np.concatenate((np.repeat(1, N), np.repeat(0, D + 1)))
    G =np.concatenate((-1*np.identity(N), np.transpose(np.dot(np.identity(N) * l, np.transpose(X))), np.transpose(-1*l)))
    G= np.transpose(G)
    h = np.repeat(-1, G.shape[1]) 
    A = np.identity(n = N + D + 1)
    b= np.repeat(1, N + D + 1)
    
    P = matrix(P.astype('float'))
    q = matrix(q.astype('float'))
    G = matrix(G.astype('float'))
    h = matrix(h.astype('float'))
    A = matrix(A.astype('float'))
    b = matrix(b.astype('float'))

    sol = cvxopt.solvers.qp(P,q,G, h, A, b)
    #http://cvxopt.org/userguide/coneprog.html#quadratic-programming
    
  
    #construct H, f, A, b, and lb
    #quadprog.solve_qp()
        #min 1/2 (x.T G X + ax)
        #st c x <= b
    sol=solvers.qp(Q, p, h)
    
    #sol = quadprog.solve_qp(G, a, c, b, meq)

# distribute components of x into w, b, and xi:

    return(w, b, xi)


# 2

In [10]:
X = mat["X"]
l = mat["L"]

    D,N = X.shape

    x = np.repeat(1, N + D + 1) #should it be 1? i honestly dont know
    #G = np.identity(n=N+D+1) * np.concatenate((np.repeat(0.00001, N), np.repeat(1, D), np.repeat(0.00001,1)), axis = 0)
    P = np.identity(n=N+D+1) * np.concatenate((np.repeat(0, N), np.repeat(1, D), np.repeat(0,1)), axis = 0)
    q = np.concatenate((np.repeat(1, N), np.repeat(0, D + 1)))
    G =np.concatenate((-1*np.identity(N), np.transpose(np.dot(np.identity(N) * l, np.transpose(X))), np.transpose(-1*l)))
    G= np.transpose(G)
    h = np.repeat(-1, G.shape[1]) 
    A = np.identity(n = N + D + 1)
    b= np.repeat(1, N + D + 1)
    
    P = matrix(P.astype('float'))
    q = matrix(q.astype('float'))
    G = matrix(G.astype('float'))
    h = matrix(h.astype('float'))
    A = matrix(A.astype('float'))
    b = matrix(b.astype('float'))

    sol = cvxopt.solvers.qp(P, q, G, h, A, b)
    #http://cvxopt.org/userguide/coneprog.html#quadratic-programming
    
  
    #construct H, f, A, b, and lb
    #quadprog.solve_qp()
        #min 1/2 (x.T G X + ax)
        #st c x <= b
    sol=solvers.qp(Q, p, h)
    
    #sol = quadprog.solve_qp(G, a, c, b, meq)

# distribute components of x into w, b, and xi:

softsvm(X, l, 0.005)

TypeError: 'G' must be a 'd' matrix of size (7339, 7339)